In [2]:
import pandas as pd
import plotly.express as px
df=pd.read_csv("retail_sales_dataset.csv")

In [3]:
# 1. Confirmar o calcular el Total Amount
df['Calculated Total'] = df['Quantity'] * df['Price per Unit']

# 2. Normalizar la columna Age
min_age = df['Age'].min()
max_age = df['Age'].max()
df['Normalized Age'] = (df['Age'] - min_age) / (max_age - min_age) if max_age != min_age else 1  # Evitar división por cero

# 3. Normalizar la columna Total Amount
min_total = df['Total Amount'].min()
max_total = df['Total Amount'].max()
df['Normalized Total'] = (df['Total Amount'] - min_total) / (max_total - min_total) if max_total != min_total else 1

# Mostrar el DataFrame con las nuevas columnas
print(df)

     Transaction ID        Date Customer ID  Gender  Age Product Category  \
0                 1  2023-11-24     CUST001    Male   34           Beauty   
1                 2  2023-02-27     CUST002  Female   26         Clothing   
2                 3  2023-01-13     CUST003    Male   50      Electronics   
3                 4  2023-05-21     CUST004    Male   37         Clothing   
4                 5  2023-05-06     CUST005    Male   30           Beauty   
..              ...         ...         ...     ...  ...              ...   
995             996  2023-05-16     CUST996    Male   62         Clothing   
996             997  2023-11-17     CUST997    Male   52           Beauty   
997             998  2023-10-29     CUST998  Female   23           Beauty   
998             999  2023-12-05     CUST999  Female   36      Electronics   
999            1000  2023-04-12    CUST1000    Male   47      Electronics   

     Quantity  Price per Unit  Total Amount  Calculated Total  Normalized A

In [4]:
df['Calculated Total'] = df['Quantity'] * df['Price per Unit']

# Definir los límites para las categorías usando cuantiles
low_threshold = df['Calculated Total'].quantile(0.33)
high_threshold = df['Calculated Total'].quantile(0.66)

# Crear la columna de clasificación de ventas
def classify_sales(total):
    if total <= low_threshold:
        return 'Baja'
    elif total <= high_threshold:
        return 'Media'
    else:
        return 'Alta'

df['Sales Category'] = df['Calculated Total'].apply(classify_sales)

# Mostrar el DataFrame con la nueva columna de clasificación
print(df)

     Transaction ID        Date Customer ID  Gender  Age Product Category  \
0                 1  2023-11-24     CUST001    Male   34           Beauty   
1                 2  2023-02-27     CUST002  Female   26         Clothing   
2                 3  2023-01-13     CUST003    Male   50      Electronics   
3                 4  2023-05-21     CUST004    Male   37         Clothing   
4                 5  2023-05-06     CUST005    Male   30           Beauty   
..              ...         ...         ...     ...  ...              ...   
995             996  2023-05-16     CUST996    Male   62         Clothing   
996             997  2023-11-17     CUST997    Male   52           Beauty   
997             998  2023-10-29     CUST998  Female   23           Beauty   
998             999  2023-12-05     CUST999  Female   36      Electronics   
999            1000  2023-04-12    CUST1000    Male   47      Electronics   

     Quantity  Price per Unit  Total Amount  Calculated Total  Normalized A

In [7]:
# Convertir la columna 'Date' a tipo fecha y extraer el mes
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

# Agrupar por Producto y Mes y aplicar funciones de agregación
grouped_df = df.groupby(['Product Category', 'Month']).agg(
    Total_Amount_Sum=('Total Amount', 'sum'),
    Total_Amount_Mean=('Total Amount', 'mean'),
    Quantity_Count=('Quantity', 'count'),
    Min_Price=('Price per Unit', 'min'),
    Max_Price=('Price per Unit', 'max'),
    Price_Std=('Price per Unit', 'std'),
    Price_Var=('Price per Unit', 'var')
).reset_index()

# Mostrar el DataFrame agrupado con las estadísticas descriptivas
print(grouped_df)

   Product Category  Month  Total_Amount_Sum  Total_Amount_Mean  \
0            Beauty      1             13930         535.769231   
1            Beauty      2             14035         539.807692   
2            Beauty      3             10545         502.142857   
3            Beauty      4             11905         410.517241   
4            Beauty      5             12450         444.642857   
5            Beauty      6             10995         439.800000   
6            Beauty      7             16090         595.925926   
7            Beauty      8              9790         407.916667   
8            Beauty      9              6320         316.000000   
9            Beauty     10             15355         495.322581   
10           Beauty     11              9700         388.000000   
11           Beauty     12             12400         496.000000   
12         Clothing      1             13125         504.807692   
13         Clothing      2             14560         441.21212

In [8]:
# Calcular el total por venta si no está ya calculado
df['Calculated Total'] = df['Quantity'] * df['Price per Unit']

# Agrupar por Product Category y calcular la desviación de cada venta respecto a la media de su grupo
def deviation_from_mean(group):
    group['Deviation from Mean'] = group['Calculated Total'] - group['Calculated Total'].mean()
    return group

# Aplicar la función personalizada al DataFrame agrupado por 'Product Category'
df = df.groupby('Product Category').apply(deviation_from_mean)

# Mostrar el DataFrame con la nueva columna de desviación
print(df)

                      Transaction ID       Date Customer ID  Gender  Age  \
Product Category                                                           
Beauty           0                 1 2023-11-24     CUST001    Male   34   
                 4                 5 2023-05-06     CUST005    Male   30   
                 5                 6 2023-04-25     CUST006  Female   45   
                 11               12 2023-10-30     CUST012    Male   35   
                 20               21 2023-01-14     CUST021  Female   50   
...                              ...        ...         ...     ...  ...   
Electronics      988             989 2023-12-28     CUST989  Female   44   
                 991             992 2023-08-21     CUST992  Female   57   
                 992             993 2023-02-06     CUST993  Female   48   
                 998             999 2023-12-05     CUST999  Female   36   
                 999            1000 2023-04-12    CUST1000    Male   47   

           

C:\Users\feded\AppData\Local\Temp\ipykernel_9564\1124629841.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Product Category').apply(deviation_from_mean)
